#**Loading Modules**



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# importing required libraries
import numpy as np
import cv2
from matplotlib import image
from skimage.transform import resize
import os
from skimage import color
from sklearn.cluster import KMeans
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import InputLayer, Input, Concatenate
from keras.layers import Conv2D,MaxPool2D,Dense,Flatten,Dropout,GlobalAveragePooling2D, BatchNormalization
from keras.models import  Model
import tensorflow as tf
from keras.regularizers import l2
from keras import callbacks
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from sklearn.svm import SVC
from sklearn import svm
from sklearn.metrics import confusion_matrix,accuracy_score, precision_score, f1_score, recall_score,classification_report,roc_curve, auc

In [ ]:
!pip install visualkeras
import visualkeras

# **Installing opencv version 4.6.0**

In [ ]:
# !pip uninstall opencv-python==4.1.2 
#  !pip install opencv-python==4.6.0.66
cv2.__version__

# **Dataset prepation**

In [ ]:
#removing duplicate images from dataset using hashing
DATA_PATH = '/content/drive/MyDrive/Updated Dataset/'
spamData=[]
label=[]
hashList=[]
count=2
#from PIL import Image
#image.LOAD_TRUNCATED_IMAGES = True
for folder in os.listdir(DATA_PATH):
    if folder=="UpdatedHam":
      continue;
    print(">>>Reading ",folder)
    count-=1
    print(count)
    
    for file in os.listdir(DATA_PATH+folder):
         if(str(file).endswith('.jpg') or str(file).endswith('.JPG') or str(file).endswith('.jpeg') or str(file).endswith('.JPEG')):
            img = image.imread(DATA_PATH+folder+'/'+file)#open ki jagah imread
            hsh = hash(tuple(np.array(img).flatten()))
            if(hsh not in hashList):
              spamData.append(resize(img, (156, 156, 3)))
              hashList.append(hsh)
              label.append(count)
spamDataarr=np.array(spamData)
labelarr=np.array(label)


>>>Reading  UpdatedSpam
1
>>>Reading  HAM
0


/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


In [ ]:
#specifying result path
resultPath = '/content/drive/MyDrive/Result'
train_folder = os.listdir(DATA_PATH).remove("UpdatedHam")

In [ ]:
# visulaizing dataset
print("Spam data shape : ",spamDataarr.shape," Label shape : ",labelarr.shape)
print("Number of SPAM",len(labelarr[labelarr==0]))
print("Number of HAM",len(labelarr[labelarr==1]))

In [ ]:
# splitting dataset into test and train
#test_valid_Percentage = 0.3
testPercentage=0.3
x_train,x_test,y_train,y_test = train_test_split(spamDataarr,labelarr,test_size = testPercentage,random_state=50, stratify=label,shuffle=True)

In [ ]:
#checking shape of dataset
print("x_train shape : ",x_train.shape," y_train shape : ",y_train.shape)
print("x_test shape : ",x_test.shape," y_test shape : ",y_test.shape)
print("Number of train SPAM",len(y_train[y_train==0]))
print("Number of train HAM",len(y_train[y_train==1]))

print("Number of test SPAM",len(y_test[y_test==0]))
print("Number of test HAM",len(y_test[y_test==1]))

# **Shift and HOG feature extraction**

In [ ]:
# extracting grayscale dataset from rgb dataset
data_gray_train = [ color.rgb2gray(i) for i in x_train]
data_gray_test = [ color.rgb2gray(i) for i in x_test]
data_gray_train=np.array(data_gray_train)
data_gray_test=np.array(data_gray_test)
print(data_gray_train.shape)
print(data_gray_test.shape)

In [ ]:
# extracting shift features (train)
sift_features_train=[]
for image1 in data_gray_train:
  feature_extractor = cv2.SIFT_create()
  image2 = cv2.normalize(image1, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
  kp_l, desc_l = feature_extractor.detectAndCompute(image2, None)
  sift_features_train.append(desc_l)
sift_features_train.shape

In [ ]:
# extracting shift features (test)
sift_features_test=[]
for image1 in data_gray_test:
  feature_extractor = cv2.SIFT_create()
  image2 = cv2.normalize(image1, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
  kp_l, desc_l = feature_extractor.detectAndCompute(image2, None)
  sift_features_test.append(resize(desc_l, (50, 128, 1)))

In [ ]:
# resing shift features using bag of words model
center_train=[]
kmeans = KMeans(n_clusters=100, random_state=0)
for i in sift_features_train:
 (x,y)=i.shape
 if x<100:
   i=resize(i,(100,128))
   clusters = kmeans.fit_predict(i)
   center_train.append(kmeans.cluster_centers_)
 else:
   clusters = kmeans.fit_predict(i)
 #print(kmeans.cluster_centers_.shape)
   center_train.append(kmeans.cluster_centers_)

In [ ]:
# converting to numpy array
sift_features_train=np.array(sift_features_train)
sift_features_test=np.array(sift_features_test)
print(sift_features_train.shape)
print(sift_features_test.shape)

In [ ]:
#HOG feature extraction (train)
hog_image_train = []
for image in data_gray_train:
    fd, hog_image = hog(image, orientations=9, pixels_per_cell=(8, 8),
                	cells_per_block=(2, 2), visualize=True)
    hog_image_train.append(hog_image)

In [ ]:
#HOG feature extraction (test)
hog_image_test = []
for image in data_gray_test:
    fd, hog_image = hog(image, orientations=9, pixels_per_cell=(8, 8),
                	cells_per_block=(2, 2), visualize=True)
    hog_image_test.append(hog_image)

In [ ]:
# converting to numpy array
hog_image_train=np.array(hog_image_train)
hog_image_test=np.array(hog_image_test)
print(hog_image_train.shape)
print(hog_image_test.shape)

# **Model**

In [ ]:
# importing pretrained VGG19 model
vgg = VGG19(weights='imagenet', include_top=False, input_shape=(156, 156, 3))
vgg.trainable= False
vgg.summary()

In [ ]:
# Top stream model
inp1= Input((156,156,3))
x1=vgg(inp1)
x1=Flatten()(x1)
x1 = Dense(512,activation='relu')(x1)
x1 = Dropout(0.3)(x1)
x1 = Dense(128,activation='relu')(x1)
x1 = Dropout(0.3)(x1)
x1 = BatchNormalization()(x1)
out1 = Dense(1, activation='sigmoid')(x1)
m1 = Model(inp1, out1)

m1.summary()

In [ ]:
# visualizing model
visualkeras.layered_view(model_final, legend=True)

In [ ]:
# Middle stream model
inp2 = Input((156,156,1))
x2=Conv2D(32,kernel_size=3, padding='same',activation='relu')(inp2)
x2=MaxPool2D(pool_size=(3, 3))(x2)
x2=Conv2D(64,kernel_size=3, padding='same',activation='relu')(x2)
x2=MaxPool2D(pool_size=(3, 3))(x2)
x2=Conv2D(128,kernel_size=3, padding='same',activation='relu')(x2)
x2=Dropout(0.1)(x2)
x2=Flatten()(x2)
x2 = Dense(512,activation='relu')(x2)
x2 = Dropout(0.3)(x2)
x2 = Dense(128,activation='relu')(x2)
x2 = Dropout(0.3)(x2)
x2 = BatchNormalization()(x2)
out2 = Dense(1, activation='sigmoid')(x2)
m2 = Model(inp2, out2)
m2.summary()

In [ ]:
# Bottom stream model
inp3= Input((50,128,1))
x3=Conv2D(32,kernel_size=3, padding='same',activation='relu')(inp3)
x3=MaxPool2D(pool_size=(2, 2))(x3)
x3=Conv2D(64,kernel_size=3, padding='same',activation='relu')(x3)
x3=MaxPool2D(pool_size=(2, 2))(x3)
x3=Conv2D(128,kernel_size=3, padding='same',activation='relu')(x3)
x3=Dropout(0.1)(x3)
x3=Flatten()(x3)
x3 = Dense(512,activation='relu')(x3)
x3 = Dropout(0.3)(x3)
x3 = Dense(128,activation='relu')(x3)
x3 = BatchNormalization()(x3)
out3 = Dense(1, activation='sigmoid')(x3)
m3 = Model(inp3, out3)

m3.summary()

In [ ]:
#concatenating feature stream from all three model

concat = Concatenate()([m1.layers[5].output, m2.layers[10].output, m3.layers[10].output]) 
# merge = Model([m1.input, m2.input,m3.input], concat)
m = Dense(384, activation='relu')(concat)#is 384 taken randomly
m = Dropout(0.3)(m)
m = Dense(128, activation='relu')(m)
m = Dropout(0.1)(m)
m = Dense(1, activation='sigmoid')(m)

model_final = Model(inputs=[m1.input, m2.input, m3.input], outputs=m)
model_final.summary()



In [ ]:
#compiling model
model_final.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

# **Training**

In [ ]:
#model training
# NO_OF_EPOCHS=20
# BATCH_SIZE=32
# checkpointer = callbacks.ModelCheckpoint(filepath=resultPath+"/final_model/checkpoint-{epoch:04d}.hdf5", verbose=1, save_best_only=True, monitor='val_accuracy',mode='max')
# csv_logger = CSVLogger(resultPath+'/result_logger.csv',separator=',', append=False)
# reduce_lr = ReduceLROnPlateau(monitor='accuracy', factor=0.2, patience=2, min_lr=0.001)
# model_final.fit([x_train,hog_image_train,sift_features_train],y_train,epochs=NO_OF_EPOCHS,verbose=1,batch_size=BATCH_SIZE,validation_data=([x_test,hog_image_test,sift_features_test],y_test),callbacks=[checkpointer,csv_logger,reduce_lr])

# **Testing**

In [ ]:
#model testing
model_final.load_weights(resultPath + "/final_model/checkpoint-0009.hdf5")
prediction_prob1 = model_final.predict([x_test,hog_image_test,sift_features_test],verbose=1)
y_pred=np.round(prediction_prob1)
print(accuracy_score(y_test,y_pred))
print(precision_score(y_test,y_pred,average='binary'))
print(recall_score(y_test,y_pred,average='binary'))
print(f1_score(y_test,y_pred,average='binary'))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

# **Adding SVM**

In [ ]:
# defining submodel
m1model = Model(inputs=model_final.input,outputs=model_final.get_layer('dense_10').output)

In [ ]:
#training submodel
m1_x_train = m1model.predict([x_train,hog_image_train,sift_features_train],verbose=1)
m1_x_test = m1model.predict([x_test,hog_image_test,sift_features_test],verbose=1)
print("Shape of model1 Train and Test DF : ",m1_x_train.shape," : ",m1_x_test.shape)

In [ ]:
#defining accuracy matrix
def printMetrics(true,pred):
    print("Accuracy : ",accuracy_score(true, pred))
    print("Precision",precision_score(true, pred , average="weighted"))
    print("Recall : ",recall_score(true, pred , average="weighted"))
    print("F1-score : ",f1_score(true, pred, average="weighted"))
    print("Confusion Matrix : ")
    print(confusion_matrix(true, pred))
    print(classification_report(true,pred))

In [ ]:
#predicting output after adding SVM
RSVM = svm.SVC(kernel='rbf',probability=True)
RSVM.fit(m1_x_train, y_train)
RSVMprob = RSVM.predict_proba(m1_x_test)
y_pred = RSVM.predict(m1_x_test)
printMetrics(y_test,y_pred)